This is an implementation of Ant Colony Optimization to solve a Travelling Salesman Problem.

This part imports and prepares data

In [121]:
#import all necessary modules
import os
import numpy as np
import math as mth
import random as rnd
rnd.seed=1234

#this will be our global optimum objective function
global_best_obj_fun = 0
#this will be our global optimum path from a source node
global_best_path = []

dir = os.getcwd()
file_name = os.path.join(dir, 'sub_data_file.csv')
data = np.loadtxt(file_name, delimiter=',', usecols=[1,2])
bases_arr = np.array([[-5000,5000], #Lyndhurst coords
                      [5000,-5000]]) #Beaulieu coords
#add the Lyndhurst and Beaulieu bases
data2 = np.append(data, bases_arr, axis=0) #data 2 is the data with 2 bases attached at the end
#print(data2.shape)

#define Euclidean distance
def dist(x1, x2, y1, y2):
    return mth.sqrt((x2-x1)**2 + (y2-y1)**2)
    
#make a matrix of distance
dist_matrix = np.zeros([152, 152])
#print(dist_matrix)
for a_key, a_coord in enumerate(data2):
    for b_key, b_coord in enumerate(data2):
        dist_matrix[a_key, b_key] = dist(a_coord[0], b_coord[0], a_coord[1], b_coord[1])
#print(dist_matrix[:5, :])

trans_rates_matrix = np.zeros([152,152])
#print(trans_rates_matrix)

def trans(dist):
    if dist<3000 and dist >=2500: return 1
    elif dist<2500 and dist>=2000: return 2
    elif dist<2000 and dist>=1500: return 3
    elif dist<1500 and dist>=1000: return 4
    elif dist<1000 and dist>=500: return 5
    elif dist<500 and dist>0: return 7
    else: return 0
            
for ind_row, row in enumerate(dist_matrix):
    for ind_col, col in enumerate(dist_matrix):
        trans_rates_matrix[ind_row, ind_col] = trans(dist_matrix[ind_row, ind_col])
#trans_rates_matrix[-10:,-10:]

Defining the problem

We need to find optimal routes which maximise the mult-objective function as defined below.
The latency is adjusted accordingly in order to become a reciprocal, therefore a single goal can be achieved; here: maximisation.
beta has been set as default to 150 as this equalises the numbers between end-to-end transfer and latency, therefore differences in both remain computationally relevant for the objective function.

In [122]:
def obj_fun(transfer, latency, alpha=1, beta=150):
    return ((alpha*transfer) + (beta/latency))

Initiate the necessary data structures: routes, pheromone_matrix 

In [123]:
routes = [] # solutions for each node
pheromone_matrix = np.zeros([152,152])
#pheromone_matrix.shape

Defining algorithm's control parameters

In [124]:
ants_n = 20 #number of ants
iters = 10 #iterations of algorithm till stop
evap_rate=0.45 #evaporation rate (45%)


We want to create a list of indices that will contain all nodes with the indices of their incident nodes (Mbps >0)

In [125]:
out_edges = list([])
current_edges =list([])

for key1, i in enumerate(trans_rates_matrix):
    for key2, j in enumerate(trans_rates_matrix):
        if trans_rates_matrix[key1,key2] > 0:
            current_edges.append(key2)
    out_edges.append({key1 : current_edges})
    current_edges= [] 
print(out_edges)


[{0: [7, 19, 39, 41, 54, 75, 85, 92, 93, 99, 113, 126, 132, 145, 149]}, {1: [3, 10, 11, 12, 37, 46, 47, 66, 70, 74, 81, 82, 87, 127, 131, 136, 146]}, {2: [8, 21, 22, 23, 25, 32, 38, 43, 49, 50, 53, 55, 67, 68, 69, 79, 98, 103, 107, 109, 110, 118, 121, 122, 124, 125, 140, 141, 143, 150]}, {3: [1, 10, 11, 12, 26, 37, 46, 65, 66, 72, 74, 77, 81, 87, 104, 127, 136, 139, 142]}, {4: [15, 21, 25, 30, 31, 32, 33, 38, 43, 49, 50, 55, 59, 62, 64, 68, 79, 98, 101, 103, 107, 108, 110, 116, 118, 120, 122, 124, 125, 140, 141, 143]}, {5: [9, 16, 19, 20, 23, 28, 34, 39, 41, 51, 53, 54, 58, 67, 75, 84, 85, 86, 92, 93, 95, 99, 100, 105, 109, 113, 114, 126, 132, 133, 134, 135, 147, 149]}, {6: [9, 15, 17, 18, 21, 23, 24, 25, 28, 29, 32, 33, 34, 38, 44, 45, 49, 57, 63, 64, 67, 78, 79, 84, 86, 88, 90, 94, 97, 100, 101, 105, 106, 109, 112, 115, 116, 118, 120, 121, 129, 137, 138, 140, 142, 143, 147]}, {7: [0, 19, 39, 41, 54, 75, 85, 92, 93, 99, 113, 126, 132, 135, 145, 149]}, {8: [2, 15, 16, 20, 21, 22, 23, 2

Time to populate the pheromone matrix with random initial values 

In [126]:
for i in range(152):
    for j in range(i,152):
        if i != j and trans_rates_matrix[i, j] > 0 :
            phero = rnd.random()
            while phero <= 1/150 or phero >= 0.5: #let's make sure that each edge has some appreciable pheromone on it but not too much 
                phero = rnd.random()
            pheromone_matrix[i,j] = pheromone_matrix[j,i] = phero
print(pheromone_matrix)

[[0.         0.         0.         ... 0.24762565 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.05760665 0.        ]
 ...
 [0.24762565 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.05760665 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


Let's make a pheromone list of dicts to mirror out_edges ones, it's easier to code

In [127]:
out_pheros = []
def out_pheros_update():
    """
    converts pheromone matrix to a dictionary of a node with exiting pheromone levels in the order corresponding to the out_edges indices
    """
    global out_pheros
    out_pheros = []
    #loop to populate
    for key1, i in enumerate(trans_rates_matrix):
        current_pheros = [] 
        for key2, j in enumerate(trans_rates_matrix):
            #only append pheromone if connection exists >> Mbps >0
            if  trans_rates_matrix[key1, key2] > 0:
                phero = pheromone_matrix[key1, key2]
                if phero != 0 : current_pheros.append(phero) # not sure why but the algo was appending zeros!
         # Only append if current_pheros is not empty
        if current_pheros:  # Check if there are any pheromone values collected
            out_pheros.append({key1: current_pheros})  # Append the dictionary for the current node  
            
out_pheros_update()
print(out_pheros)

[{0: [np.float64(0.37662584479793615), np.float64(0.1868410271629024), np.float64(0.38731604744682346), np.float64(0.2727442576367759), np.float64(0.13800546877733644), np.float64(0.19597463212233435), np.float64(0.38291958711899876), np.float64(0.13299536109180699), np.float64(0.42664425792546634), np.float64(0.16296171971393258), np.float64(0.3359189122884646), np.float64(0.34659409219117976), np.float64(0.2687751787755611), np.float64(0.20699350859724153), np.float64(0.24762564648988628)]}, {1: [np.float64(0.008262040154973338), np.float64(0.10763374893935063), np.float64(0.08270201647636988), np.float64(0.41053695679434077), np.float64(0.016561281079859103), np.float64(0.2636354557421765), np.float64(0.08289133485184574), np.float64(0.050259763805114877), np.float64(0.4165133450808044), np.float64(0.26440130232616255), np.float64(0.38818855944896025), np.float64(0.4733711670802945), np.float64(0.2756936579168089), np.float64(0.1993895653693395), np.float64(0.2872348095630114), np.f

Creating routes data structure for ant for each iteration
Creating the best rout data structure with corresponding obj_fun value

In [128]:
def ACO_ant_trans(obj_fun, out_edges, out_pheros, source, dest, alpha=1, beta=0.4):
    if source!=dest:
        ant_pos=source
    else: return dest #done! we got there! nothing to do!
    
    nx_edg_attract = []
    
    next_edges = out_edges[ant_pos]
    for edge in next_edges[ant_pos]: #this one was a pain in the adverbial, I had to put the index or it would return whole lists of next edges !!!
       edge_obj_fun = obj_fun(trans_rates_matrix[ant_pos, edge], 30)
       nx_edg_attract.append(edge_obj_fun) 
    
    nx_edg_attract_beta = [x**beta for x in nx_edg_attract]
    next_pheros = out_pheros[ant_pos] # all edges out of ant_pos
    next_pheros_ant_pos_alpha = [x ** alpha for x in next_pheros[ant_pos]] #all pheromones in adj node go up to power alpha as per ant's prob of trans equation
    phero_attract = zip(next_pheros_ant_pos_alpha, nx_edg_attract_beta)
    phero_attract_multipl = [a*b for a,b in phero_attract]
    prob_denominator = sum(phero_attract_multipl)
    prob_numerators = [a * b for a,b in zip(next_pheros_ant_pos_alpha, nx_edg_attract_beta)] #tau^alpha * eta^beta where eta is Greek elongated n - top of the equation
    trans_prob = []
    for prob_numerator in prob_numerators:
        trans_prob.append(prob_numerator / prob_denominator)
    roulette = [] #list of roulette 'slots'
    cum_sum=0
    roulette.append(0)
    for prob in trans_prob:
        cum_sum += prob
        roulette.append(cum_sum)
    roulette[-1] = 1 # python float calcs are so 'sh*t'
    roulette_spin = rnd.random()
    next_edge = -999 #'invalid' value
    for slot, val in enumerate(roulette):
        if roulette_spin > val: 
            next_edge = slot # which slot did our spin value land in?
        else: break #if it landed in a slot, finish the show and don't check all other slots, save comp power
    return next_edges[ant_pos][next_edge] # give us back your choice of next edge then as an index int

def remove_loops(ant_path):
    """
    Removes duplicate nodes and intermediary nodes - in cycles from an ant path.
    Args:
        ant_path (list): A list of nodes representing the ant path.
    Returns:
        list: A list of nodes with duplicates and cycles removed.
    """
    pops = 0 # number of the list pops to reduce obj fun after loop removal - latency will go down!
    visited = set()  # Tracks visited nodes, here are only unique values of nodes
    result = []  # Final path without duplicates or cycles
    for node in ant_path:
        if node in visited:
            # If a cycle starts, remove nodes from result until the cycle's start node is removed
            while result and result[-1] != node: #while result is truthy = exists and we have not encountered the visted node when popping bits and bobs off the end
                visited.remove(result.pop()) #pop() removes the last element and returns it at the same time
                pops += 1
        else:
            # Add new node to the result and mark it as visited
            result.append(node)
            visited.add(node)
    return result, pops #clean simple path, with pops count for latency change for obj fun update

def ACO_ant_path(obj_fun, out_edges, out_pheros, source, dest, alpha=1, beta=0.4):
#let's find a path from node source (e.g. )147 to node dest (e.g. 150 - Lyndhurst)
    edge_trans_rate = np.inf
    ant_path_obj_fun = 0
    ant_path_latency = 0
    ant_path = []
    first=next = source
    ant_path.append(next)
    while next != dest and len(ant_path) <= 151: #the ant has not reached BS index dest - 150 or 151 (Lyndhurst, Beaulieu, respectively); if the ant got lost after 150 edge transitions, forget it! Introduced to remove program hang-ups...
        old_saved = next
        next=ACO_ant_trans(obj_fun, out_edges, out_pheros, next, dest)
        ant_path.append(next)
        ant_path_latency += 30
        if trans_rates_matrix[old_saved, next] < edge_trans_rate: #save the minimum transmission rate as encountered
            edge_trans_rate = trans_rates_matrix[old_saved, next] 
    else: last=next #after all done and dusted, this is where the while-condition proved false :) 
    ant_path_obj_fun = obj_fun(edge_trans_rate, ant_path_latency)
    simple_ant_path, nodes_popped = remove_loops(ant_path)
    simple_path_latency = ant_path_latency - nodes_popped
    simple_ant_path_obj_fun = obj_fun(edge_trans_rate, simple_path_latency ) #we are ASSUMING popping the nodes in a cycle did not change the obj fun due to increased transfer - checking it would be just complex and fairly redundant! Ants will find the way
    #print(f"Simple path from node {first} to {last}: {simple_ant_path}; \nwhere: 150 - Lyndhurst, 151 - Beauleu\nMiminum transfer on route: {edge_trans_rate}MBps, \nlatency on route : {simple_path_latency} ms\nAnt path's objective function (multi): {ant_path_obj_fun}\nAnt path's obj fun after dupl popping : {simple_ant_path_obj_fun}")
    return simple_ant_path, simple_ant_path_obj_fun

def evaporate(evap_rate=0.4):
    global pheromone_matrix
    a = 1 - evap_rate
    pheromone_matrix *= a
    
def global_phero_update(best_path, best_obj_fun, evap_rate=0.4):
    #print(best_path)
    #print(best_obj_fun)
    evaporate(evap_rate)
    global pheromone_matrix
    phero_to_drop = 1.5 * best_obj_fun / len(best_path)
    #print(f"phero to drop on each edge: {phero_to_drop}")
    for i in best_path:
        for j in best_path:
            pheromone_matrix[i,j] += phero_to_drop
            pheromone_matrix[j,i] += phero_to_drop
            
#we make the algo iterations from a source node function here where we release ants_n=30 ants in to the wild
def ACO_iter(obj_fun, out_edges, out_pheros, source, alpha=1, beta=1, ants_n = 20, evap_rate = 0.45, iters =10):

    global global_best_path
    global global_best_obj_fun
    best_path = [] # for this iteration!
    best_obj_fun = 0
    #where the ant starts
    source = source
    paths_to_any_base = [] # literally all paths found by 30 ants in this iteration, however ridiculous
    
    for i in range(ants_n - 1):
        dest = rnd.choice([150,151])
        ant_path, ant_path_obj_fun = ACO_ant_path(obj_fun, out_edges, out_pheros, source, dest)
        paths_to_any_base.append(ant_path)
        if ant_path_obj_fun > best_obj_fun:
            best_obj_fun = ant_path_obj_fun
            best_path = ant_path

    global_phero_update(best_path, best_obj_fun)
    out_pheros_update()
  
    #print(f"\nBest path: {best_path}\nBest objective function: {best_obj_fun}")
    if best_obj_fun > global_best_obj_fun:
        global_best_obj_fun = best_obj_fun 
        global_best_path = best_path

#this functions is doing the given number of iterations for a given source - the main algorithm
def ACO(obj_fun, out_edges, out_pheros, source):
    for i in range(iters-1):
#        print(f"Iteration {i+1} of {iters}:  {ants_n} ants are finding their path...")
        ACO_iter(obj_fun, out_edges, out_pheros, source)


for source in range(150):
    #running commentary to see the algo has not got stuck or something
    global global_best_path
    global global_best_obj_fun
 #   print(f"Starting algo for source = {source}")
    ACO(obj_fun, out_edges, out_pheros, source)
 #   print(f"Completed algo for source = {source}. Found route to a Base Station\nPath: {global_best_path}\nObjective function value: {global_best_obj_fun}\n")
    routes.append({"source": source,
               "best_route": global_best_path, 
               "objective function": global_best_obj_fun})
    global_best_path = [] #we need to clear these both for each source algorithm run; global means for a singular run of the algorithm e.g. find path from node 130 > node 151
    global_best_obj_fun = 0 #as mentioned ^^^

#output the solutions file to screen
#print("\n\n\n\n\n\n\n\n\n\nALL ROUTES AND DATA FOR SOLUTIONS FILE:\n\n")
#print(routes)
#write the solutions file to hard disk
with open('solutions.txt','w') as file:
    file.write(str(routes))


Starting algo for source = 0
Iteration 1 of 10:  20 ants are finding their path...
Iteration 2 of 10:  20 ants are finding their path...
Iteration 3 of 10:  20 ants are finding their path...
Iteration 4 of 10:  20 ants are finding their path...
Iteration 5 of 10:  20 ants are finding their path...
Iteration 6 of 10:  20 ants are finding their path...
Iteration 7 of 10:  20 ants are finding their path...
Iteration 8 of 10:  20 ants are finding their path...
Iteration 9 of 10:  20 ants are finding their path...
Completed algo for source = 0. Found route to a Base Station
Path: [0, 7, 99, 92, 85, 23, 143, 50, 150]
Objective function value: 1.625

Starting algo for source = 1
Iteration 1 of 10:  20 ants are finding their path...
Iteration 2 of 10:  20 ants are finding their path...
Iteration 3 of 10:  20 ants are finding their path...
Iteration 4 of 10:  20 ants are finding their path...
Iteration 5 of 10:  20 ants are finding their path...
Iteration 6 of 10:  20 ants are finding their pat